In [19]:
# === Cell 1: Unified Environment & Project-Wide Setup ===
import os, json, math, datetime as dt
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
os.makedirs('outputs', exist_ok=True)

# Optional (map preview)
try:
    import geemap
    GEEMAP_AVAILABLE = True
except Exception:
    GEEMAP_AVAILABLE = False

import ee

# ----- Earth Engine init (force a registered EE project) -----
EE_PROJECT_ID = os.environ.get('EE_PROJECT_ID', 'nasa-flood')  # ★ 본인 EE 프로젝트로!

def _ee_init(project_id: str) -> str:
    """Initialize Earth Engine with explicit project."""
    try:
        # 바로 지정 프로젝트로 초기화 시도
        ee.Initialize(project=project_id)
        return f"✅ Initialized with project='{project_id}'"
    except Exception:
        # 인증 후 재시도
        print("🔐 Authenticating with Earth Engine...")
        ee.Authenticate()
        ee.Initialize(project=project_id)
        return f"✅ Authenticated & initialized with project='{project_id}'"

print(_ee_init(EE_PROJECT_ID))
print(f"⏰ Current time: {datetime.now().isoformat(timespec='seconds')}")

# ===== Project-wide constants =====
CFG = {
    # AOIs (EPSG:4326) - Geodesic=False for stability in reduce operations
    "AOI_DELTA": ee.Geometry.Rectangle([104.30,  8.50, 106.90, 10.90], geodesic=False),
    "AOI_TONLESAP": ee.Geometry.Rectangle([103.30, 12.00, 105.20, 13.70], geodesic=False),

    # Analysis windows
    "YEARS": list(range(2015, 2025)),
    "FLOOD_MONTHS": (8, 9),     # Aug–Sep (wet-season peak)
    "DROUGHT_MONTHS": (3, 4),   # Mar–Apr (dry-season trough)

    # Thresholds (empirical values from literature)
    # Reference: Twele et al. (2016) - Sentinel-1 flood mapping
    "TH_VV_DB": -16.0,  # Open water (specular reflection)
    "TH_VH_DB": -22.0,  # Flooded vegetation (double-bounce)

    # Landsat5 baseline window (pre-major-dam reference)
    "BASELINE_YEARS": [2005, 2006, 2007, 2008],

    # Event markers for plots
    "EVENTS": {
        "JINGHONG_FLOW_CUT": "2019-07-15",  # Smoking gun event
        "XIAOWAN_ONLINE":    "2009-01-01",
        "NUOZHADU_ONLINE":   "2012-01-01"
    }
}

# ===== Robust Geometry Utilities =====
def safe_geom(g, max_error=100):
    """
    Ensure non-zero error margin geometry for topology operations.
    
    Why: GEE reduce operations fail with zero-error-margin geometries.
    max_error=100m is appropriate for 10-30m resolution analysis.
    """
    if isinstance(g, ee.Geometry):
        return g
    return ee.Feature(g).geometry(max_error)

def safe_union(geoms, max_error=100):
    """Union multiple geometries with error tolerance."""
    fc = ee.FeatureCollection([ee.Feature(gg) for gg in geoms])
    return fc.geometry(max_error)

# ===== Utilities (Date ranges) =====
def _daterange_of_year_months(year: int, m1: int, m2: int):
    """
    Return ISO start and inclusive end-of-month last day for [m1..m2].
    
    Example:
        _daterange_of_year_months(2019, 3, 4)
        → ('2019-03-01', '2019-04-30')
    """
    start = dt.date(year, m1, 1)
    if m2 == 12:
        end = dt.date(year+1, 1, 1) - dt.timedelta(days=1)
    else:
        end = dt.date(year, m2+1, 1) - dt.timedelta(days=1)
    return start.isoformat(), end.isoformat()

# ===== Sentinel-1 Utilities =====
def s1_min(aoi, start, end, pol):
    """
    Min-composite Sentinel-1 GRD over period to stabilize speckle.
    
    Min operation rationale:
    - Water has consistently low backscatter
    - Min reduces speckle while preserving water signal
    - Equivalent to "darkest pixel" composite
    """
    region = safe_geom(aoi, 100)
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(region)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode','IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
            .select(pol)
            .min()
            .clip(region))

def classify_water(img_min, pol, threshold_db):
    """
    Water = backscatter < threshold (in dB); returns self-masked binary.
    
    Physics:
    - Open water: ~-20 to -25 dB (VV)
    - Flooded veg: ~-18 to -24 dB (VH, double-bounce)
    """
    return img_min.lt(threshold_db).selfMask()

def area_km2(mask_img, aoi, scale=30, band_name=None, tile_scale=4, max_pixels=1e13):
    """
    Compute km² of a self-masked image with robust parameters.
    
    CRITICAL FIX: Auto-detect band name from selfMask() result.
    
    Args:
        mask_img: Binary mask (ee.Image)
        aoi: Area of interest (ee.Geometry)
        scale: Pixel size in meters (default 30m for Landsat-class)
        band_name: Band to measure (None = auto-detect first band)
        tile_scale: Computation tiling factor (4 = 16 tiles, prevents memory errors)
        max_pixels: Maximum pixels to process (1e13 ~ 10 million km² at 30m)
    
    Returns:
        Area in km² (float when .getInfo() called)
    """
    region = safe_geom(aoi, 100)
    
    # Auto-detect band name (selfMask() preserves original band name)
    if band_name is None:
        band_name = ee.String(mask_img.bandNames().get(0))
    
    area_img = mask_img.multiply(ee.Image.pixelArea())
    result = area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=scale,
        maxPixels=max_pixels,
        tileScale=tile_scale  # Critical for large AOIs
    )
    return ee.Number(result.get(band_name)).divide(1e6)

# ===== Landsat5 Utilities =====
def landsat5_c2_sr_mask_scale(img):
    """
    Landsat 5 Collection 2 Level-2 scaling + cloud/shadow masking.
    
    Reference: USGS Landsat Collection 2 documentation
    - Optical bands: DN * 0.0000275 - 0.2 → Reflectance
    - Thermal band: DN * 0.00341802 + 149.0 → Kelvin
    - QA_PIXEL bits 3,4: Cloud, Cloud shadow
    """
    qa = img.select('QA_PIXEL')
    cloud  = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(shadow).eq(0))
    
    # Apply scale factors
    optical = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = img.select('ST_B6').multiply(0.00341802).add(149.0)
    
    return (img.addBands(optical, None, True)
               .addBands(thermal, None, True)
               .updateMask(mask))

def landsat5_median(aoi, start_date, end_date, months=None):
    """
    Median composite with optional wrapped-month filter.
    
    Args:
        months: Tuple of tuples for season wrapping
                Example: ((11,12),(1,4)) for Nov-Dec OR Jan-Apr (dry season)
    """
    region = safe_geom(aoi, 100)
    col = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
           .filterBounds(region)
           .filterDate(start_date, end_date)
           .map(landsat5_c2_sr_mask_scale))
    
    if months:
        # Handle season wrapping (e.g., Nov-Dec-Jan-Feb)
        f = ee.Filter.Or(
            ee.Filter.calendarRange(int(months[0][0]), int(months[0][1]), 'month'),
            ee.Filter.calendarRange(int(months[1][0]), int(months[1][1]), 'month')
        )
        col = col.filter(f)
    
    return col.median().clip(region)

def mndwi(img):
    """
    Modified Normalized Difference Water Index.
    
    Formula: (Green - SWIR1) / (Green + SWIR1)
    Range: -1 to +1 (water typically > 0)
    
    Reference: Xu (2006) - Modification of NDWI for water extraction
    """
    return img.normalizedDifference(['SR_B2','SR_B5']).rename('MNDWI')

def water_mask_from_mndwi(img, threshold=0.0):
    """Binary water mask from MNDWI (threshold typically 0.0 to 0.3)."""
    return img.gt(threshold).selfMask()

def chirps_sum_mm(aoi, start, end):
    """
    Return AOI-mean of CHIRPS precipitation sum (mm) over [start,end].
    
    CHIRPS 'precipitation' band unit: mm/day
    - .sum() → cumulative mm over period
    - .reduceRegion(mean) → spatial average across AOI
    
    Reference: UCSB Climate Hazards Group CHIRPS dataset
    """
    region = safe_geom(aoi, 100)
    col = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
           .filterBounds(region)
           .filterDate(start, end)
           .select('precipitation'))
    
    total = col.sum().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=5000,  # CHIRPS native resolution
        maxPixels=1e12,
        tileScale=4
    )
    return ee.Number(total.get('precipitation'))

# ===== Safe Prints (no server round-trips) =====
print("📍 AOI_DELTA bounds: [104.30,  8.50, 106.90, 10.90]")
print("📍 AOI_TONLESAP bounds: [103.30, 12.00, 105.20, 13.70]")
print("✅ Setup complete — All utilities loaded")

✅ Initialized with project='nasa-flood'
⏰ Current time: 2025-10-05T08:40:16
📍 AOI_DELTA bounds: [104.30,  8.50, 106.90, 10.90]
📍 AOI_TONLESAP bounds: [103.30, 12.00, 105.20, 13.70]
✅ Setup complete — All utilities loaded


In [20]:
# === Cell 2: Baseline (2005–2008) with Landsat5 + JRC cross-check ===
"""
Objective:
  - Establish pre-dam (2005–2008) wet/dry-season baselines using Landsat5
  - Cross-validate with JRC Global Surface Water permanent water layer
  - Quantify baseline water extent for Delta & Tonlé Sap

Why 2005-2008?
  - Before major dam operations (Xiaowan 2009, Nuozhadu 2012)
  - Landsat5 operational and good quality
  - Sufficient temporal depth for seasonal composites

Deliverables:
  - outputs/baseline_areas.csv (quantitative table)
  - outputs/baseline_summary.json (metadata for dashboard)
  - outputs/*.tif (geospatial layers for visualization)
"""

import warnings
warnings.filterwarnings('ignore')

# 1) Define baseline period & combined AOI
baseline_start = f"{min(CFG['BASELINE_YEARS'])}-01-01"
baseline_end   = f"{max(CFG['BASELINE_YEARS'])}-12-31"

# Union AOIs for batch processing (with safe geometry)
AOI_ALL = safe_union([CFG['AOI_DELTA'], CFG['AOI_TONLESAP']], max_error=100)

print(f"📅 Baseline period: {baseline_start} to {baseline_end}")
print(f"🌍 Combined AOI union computed")

# 2) Landsat5 Collection 2 preparation
print("🛰️  Loading Landsat5 Collection 2 Level-2...")
l5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
      .filterBounds(AOI_ALL)
      .filterDate(baseline_start, baseline_end)
      .map(landsat5_c2_sr_mask_scale))

# Check collection size
l5_count = l5.size().getInfo()
print(f"   ✓ Found {l5_count} Landsat5 scenes")

if l5_count < 10:
    print("   ⚠️  WARNING: Low scene count may affect baseline quality")

# 3) Seasonal composites
print("🌦️  Computing seasonal composites...")

# WET SEASON: May–Oct (monsoon peak, consistent across SE Asia)
l5_wet = l5.filter(ee.Filter.calendarRange(5, 10, 'month')).median().clip(AOI_ALL)

# DRY SEASON: Nov–Dec + Jan–Apr (wrapping year boundary)
# Use Filter.Or for month wrapping
dry_filter = ee.Filter.Or(
    ee.Filter.calendarRange(11, 12, 'month'),
    ee.Filter.calendarRange(1, 4, 'month')
)
l5_dry = l5.filter(dry_filter).median().clip(AOI_ALL)

print("   ✓ Wet-season composite: May–Oct median")
print("   ✓ Dry-season composite: Nov–Apr median")

# 4) MNDWI-based water masks
print("💧 Extracting water masks (MNDWI > 0)...")
mndwi_wet = mndwi(l5_wet)
mndwi_dry = mndwi(l5_dry)

# Threshold: MNDWI > 0 is conservative for water (Xu 2006)
water_wet_mask = water_mask_from_mndwi(mndwi_wet, threshold=0.0).rename('water')
water_dry_mask = water_mask_from_mndwi(mndwi_dry, threshold=0.0).rename('water')

print("   ✓ Water masks generated")

# 5) JRC Global Surface Water cross-validation
print("🔍 Cross-validating with JRC Global Surface Water...")
jrc = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').clip(AOI_ALL)

# JRC 'seasonality' band: 12 = permanent water (wet every month)
jrc_perm = jrc.select('seasonality').eq(12).selfMask().rename('perm')

print("   ✓ JRC permanent water layer loaded (seasonality=12)")

# 6) Area calculations with error handling
def _num(x):
    """Safe number extraction with None handling."""
    if x is None:
        return None
    try:
        return float(ee.Number(x).getInfo())
    except Exception as e:
        print(f"   ⚠️  Error in getInfo(): {e}")
        return None

def aoi_areas(name, aoi):
    """
    Compute baseline areas for a single AOI.
    
    Returns:
        dict with wet/dry baseline areas + JRC permanent water area
    """
    print(f"   ⏳ Computing {name}...")
    
    # Wet-season baseline
    wet_km2 = _num(area_km2(water_wet_mask, aoi, scale=30, band_name='water'))
    
    # Dry-season baseline
    dry_km2 = _num(area_km2(water_dry_mask, aoi, scale=30, band_name='water'))
    
    # JRC permanent water (for validation)
    perm_km2 = _num(area_km2(jrc_perm, aoi, scale=30, band_name='perm'))
    
    return {
        "aoi": name,
        "baseline_wet_km2": wet_km2,
        "baseline_dry_km2": dry_km2,
        "jrc_perm_km2": perm_km2
    }

print("📊 Computing baseline areas for each AOI...")
summary_rows = [
    aoi_areas("Mekong_Delta", CFG['AOI_DELTA']),
    aoi_areas("Tonle_Sap", CFG['AOI_TONLESAP']),
    aoi_areas("Union_ALL", AOI_ALL),
]

# Create summary dataframe
df_baseline = pd.DataFrame(summary_rows)

# Display with formatting
print("\n" + "="*60)
print("BASELINE AREAS SUMMARY (2005-2008)")
print("="*60)
display(df_baseline.round(1))
print("="*60)

# 7) Validation checks
print("\n🔬 Quality checks:")
for _, row in df_baseline.iterrows():
    if row['aoi'] == 'Union_ALL':
        continue
    
    # Check: Wet > Dry (sanity check)
    if row['baseline_wet_km2'] > row['baseline_dry_km2']:
        print(f"   ✓ {row['aoi']}: Wet > Dry (expected)")
    else:
        print(f"   ⚠️  {row['aoi']}: Wet ≤ Dry (unexpected!)")
    
    # Check: JRC perm water should be < dry baseline
    if row['jrc_perm_km2'] < row['baseline_dry_km2']:
        print(f"   ✓ {row['aoi']}: JRC perm < Dry baseline (expected)")
    else:
        print(f"   ℹ️  {row['aoi']}: JRC perm ≥ Dry baseline (check data)")

# 8) Save outputs
print("\n💾 Saving outputs...")
os.makedirs('outputs', exist_ok=True)

# CSV table
df_baseline.to_csv('outputs/baseline_areas.csv', index=False)
print("   ✓ outputs/baseline_areas.csv")

# JSON metadata (for dashboard)
baseline_metadata = {
    "baseline_years": CFG['BASELINE_YEARS'],
    "wet_months": [5, 10],
    "dry_months": [[11, 12], [1, 4]],
    "method": "Landsat5 C2 L2 MNDWI median composites",
    "threshold": "MNDWI > 0.0 (Xu 2006)",
    "areas": summary_rows,
    "validation": "JRC Global Surface Water v1.4 seasonality=12",
    "notes": [
        "Pre-dam reference (before Xiaowan 2009, Nuozhadu 2012)",
        "Cloud-masked using QA_PIXEL bits 3,4",
        "Median composite reduces cloud/shadow residuals"
    ]
}

with open('outputs/baseline_summary.json', 'w', encoding='utf-8') as f:
    json.dump(baseline_metadata, f, ensure_ascii=False, indent=2)

print("   ✓ outputs/baseline_summary.json")

# 9) (Optional) Export GeoTIFFs for QGIS/ArcGIS visualization
if GEEMAP_AVAILABLE:
    print("\n🗺️  Exporting GeoTIFFs (this may take 2-5 minutes)...")
    
    try:
        geemap.ee_export_image(
            mndwi_wet, 
            filename='outputs/mndwi_wet_2005_2008.tif', 
            scale=30, 
            region=AOI_ALL, 
            file_per_band=False
        )
        print("   ✓ outputs/mndwi_wet_2005_2008.tif")
        
        geemap.ee_export_image(
            mndwi_dry, 
            filename='outputs/mndwi_dry_2005_2008.tif', 
            scale=30, 
            region=AOI_ALL, 
            file_per_band=False
        )
        print("   ✓ outputs/mndwi_dry_2005_2008.tif")
        
        geemap.ee_export_image(
            water_wet_mask, 
            filename='outputs/watermask_wet_2005_2008.tif', 
            scale=30, 
            region=AOI_ALL, 
            file_per_band=False
        )
        print("   ✓ outputs/watermask_wet_2005_2008.tif")
        
        geemap.ee_export_image(
            water_dry_mask, 
            filename='outputs/watermask_dry_2005_2008.tif', 
            scale=30, 
            region=AOI_ALL, 
            file_per_band=False
        )
        print("   ✓ outputs/watermask_dry_2005_2008.tif")
        
        geemap.ee_export_image(
            jrc_perm, 
            filename='outputs/jrc_perm_seasonality12.tif', 
            scale=30, 
            region=AOI_ALL, 
            file_per_band=False
        )
        print("   ✓ outputs/jrc_perm_seasonality12.tif")
        
    except Exception as e:
        print(f"   ⚠️  GeoTIFF export failed: {e}")
        print("   ℹ️  This is non-critical; analysis data is saved in CSV/JSON")
else:
    print("\n   ℹ️  geemap not available — skipping GeoTIFF exports")

print("\n✅ BASELINE COMPUTATION COMPLETE")
print("   Next: Run notebook 02 for flood analysis (2015-2024)")

📅 Baseline period: 2005-01-01 to 2008-12-31
🌍 Combined AOI union computed
🛰️  Loading Landsat5 Collection 2 Level-2...
   ✓ Found 852 Landsat5 scenes
🌦️  Computing seasonal composites...
   ✓ Wet-season composite: May–Oct median
   ✓ Dry-season composite: Nov–Apr median
💧 Extracting water masks (MNDWI > 0)...
   ✓ Water masks generated
🔍 Cross-validating with JRC Global Surface Water...
   ✓ JRC permanent water layer loaded (seasonality=12)
📊 Computing baseline areas for each AOI...
   ⏳ Computing Mekong_Delta...
   ⏳ Computing Tonle_Sap...
   ⏳ Computing Union_ALL...

BASELINE AREAS SUMMARY (2005-2008)


,aoi,baseline_wet_km2,baseline_dry_km2,jrc_perm_km2
0,Mekong_Delta,39914.5,37503.7,22663.7
1,Tonle_Sap,4839.4,3178.6,2364.6
2,Union_ALL,44753.9,40682.3,25028.3



🔬 Quality checks:
   ✓ Mekong_Delta: Wet > Dry (expected)
   ✓ Mekong_Delta: JRC perm < Dry baseline (expected)
   ✓ Tonle_Sap: Wet > Dry (expected)
   ✓ Tonle_Sap: JRC perm < Dry baseline (expected)

💾 Saving outputs...
   ✓ outputs/baseline_areas.csv
   ✓ outputs/baseline_summary.json

🗺️  Exporting GeoTIFFs (this may take 2-5 minutes)...
Generating URL ...
An error occurred while downloading.
Total request size (1288876320 bytes) must be less than or equal to 50331648 bytes.
   ✓ outputs/mndwi_wet_2005_2008.tif
Generating URL ...
An error occurred while downloading.
Total request size (1288876320 bytes) must be less than or equal to 50331648 bytes.
   ✓ outputs/mndwi_dry_2005_2008.tif
Generating URL ...
An error occurred while downloading.
Total request size (515550528 bytes) must be less than or equal to 50331648 bytes.
   ✓ outputs/watermask_wet_2005_2008.tif
Generating URL ...
An error occurred while downloading.
Total request size (515550528 bytes) must be less than or equal to

In [21]:
# === Cell 3: Interactive Map Visualization (Optional) ===
"""
Display baseline layers on interactive map for quality control.

Only runs if geemap is available.
Useful for:
  - Visual inspection of water masks
  - Comparison of Landsat vs JRC layers
  - Identifying potential issues (clouds, shadows, misclassification)
"""

if GEEMAP_AVAILABLE:
    print("🗺️  Creating interactive map...")
    
    # Center on Tonlé Sap (between both AOIs)
    center_lat = 12.1
    center_lon = 105.2
    
    m = geemap.Map(center=[center_lat, center_lon], zoom=7)
    
    # Base layer: Landsat5 wet-season RGB (true color)
    rgb_vis = {
        'min': 0,
        'max': 0.3,
        'bands': ['SR_B3', 'SR_B2', 'SR_B1']  # RGB
    }
    m.addLayer(l5_wet, rgb_vis, 'Landsat5 Wet RGB (2005-08)')
    
    # MNDWI layer (color ramp)
    mndwi_vis = {
        'min': -1,
        'max': 1,
        'palette': ['#8c510a', '#f6e8c3', '#35978f', '#01665e']
        # Brown (land) → Teal (water)
    }
    m.addLayer(mndwi_wet, mndwi_vis, 'MNDWI Wet (2005-08)')
    
    # Water masks
    m.addLayer(
        water_wet_mask, 
        {'palette': ['#1f78b4']},  # Blue
        'Landsat Water Mask (Wet)'
    )
    
    m.addLayer(
        water_dry_mask,
        {'palette': ['#a6cee3']},  # Light blue
        'Landsat Water Mask (Dry)'
    )
    
    # JRC permanent water (for comparison)
    m.addLayer(
        jrc_perm,
        {'palette': ['#e31a1c']},  # Red
        'JRC Permanent Water (seasonality=12)'
    )
    
    # Add AOI boundaries
    delta_style = {'color': 'yellow', 'fillColor': '00000000'}
    ts_style = {'color': 'orange', 'fillColor': '00000000'}
    
    m.addLayer(CFG['AOI_DELTA'], delta_style, 'Mekong Delta AOI')
    m.addLayer(CFG['AOI_TONLESAP'], ts_style, 'Tonlé Sap AOI')
    
    # Add layer control
    m.addLayerControl()
    
    print("   ✓ Map created with 7 layers")
    print("\n📖 Layer Guide:")
    print("   • Landsat5 Wet RGB: Natural color composite (visual context)")
    print("   • MNDWI Wet: Brown=land, Teal=water (index values)")
    print("   • Landsat Water Masks: Blue=wet, Light blue=dry")
    print("   • JRC Permanent Water (Red): Validation layer")
    print("   • AOI Boundaries: Yellow=Delta, Orange=Tonlé Sap")
    print("\n   💡 Tip: Toggle layers to compare Landsat vs JRC detection")
    
    # Display map
    display(m)
    
else:
    print("ℹ️  geemap not available — skipping interactive map")
    print("   Install with: pip install geemap")
    print("   Or view outputs in QGIS using exported GeoTIFFs")

print("\n✅ Notebook 01 COMPLETE")
print("="*60)

🗺️  Creating interactive map...
   ✓ Map created with 7 layers

📖 Layer Guide:
   • Landsat5 Wet RGB: Natural color composite (visual context)
   • MNDWI Wet: Brown=land, Teal=water (index values)
   • Landsat Water Masks: Blue=wet, Light blue=dry
   • JRC Permanent Water (Red): Validation layer
   • AOI Boundaries: Yellow=Delta, Orange=Tonlé Sap

   💡 Tip: Toggle layers to compare Landsat vs JRC detection


Map(center=[12.1, 105.2], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright',…


✅ Notebook 01 COMPLETE
